In [22]:
import cv2
import numpy as np
import os
from IPython.display import clear_output


In [23]:
def dice(im1, im2):
    """
    Computes the Dice coefficient, a measure of set similarity.
    Parameters
    ----------
    im1 : array-like, bool
        Any array of arbitrary size. If not boolean, will be converted.
    im2 : array-like, bool
        Any other array of identical size. If not boolean, will be converted.
    Returns
    -------
    dice : float
        Dice coefficient as a float on range [0,1].
        Maximum similarity = 1
        No similarity = 0
        
    Notes
    -----
    The order of inputs for `dice` is irrelevant. The result will be
    identical if `im1` and `im2` are switched.
    """
    im1 = np.asarray(im1).astype(np.bool)
    im2 = np.asarray(im2).astype(np.bool)

    if im1.shape != im2.shape:
        raise ValueError("Shape mismatch: im1 and im2 must have the same shape.")

    # Compute Dice coefficient
    intersection = np.logical_and(im1, im2)

    return 2. * intersection.sum() / (im1.sum() + im2.sum())

In [2]:
def load_images_from_folder(folder,count=None,size=None):
    images = []
    i = 0
    for filename in os.listdir(folder):
        img = cv2.imread(os.path.join(folder,filename))
        if img is not None:
            i += 1
            if i%50 == 0:
                clear_output()
                print (i)
            if count is not None:
                if count == 0:
                       break
                else:
                    count -= 1
            
            if size is not None:
                img = cv2.resize(img,size,interpolation = cv2.INTER_CUBIC)
            images.append((filename,img))

    return images

In [3]:
melanoma = load_images_from_folder('./data/melanoma',None,(256,128))

350


In [4]:
others = load_images_from_folder('./data/others',None,(256,128))

1600


In [5]:
def make_melanoma_seg_dataset(data,gt_path):
    segmentation_dataset = []
    
    for filename in os.listdir(gt_path):
        for data_list in data:
            entry = [row[1] for row in data_list if row[0][:-4] in filename[:-4]]
            if entry:
                gt_img = cv2.imread(os.path.join(gt_path,filename),0)
                gt_img = cv2.resize(gt_img,(entry[0].shape[1],entry[0].shape[0]),interpolation = cv2.INTER_CUBIC)
                gt_img = gt_img.reshape((gt_img.shape[0],gt_img.shape[1],1))
                segmentation_dataset.append((entry[0],gt_img))

    return segmentation_dataset
        
        

In [7]:
dataset = make_melanoma_seg_dataset([melanoma,others],'./data/gt/')

In [8]:
#Splitting for training and testing
train_size = int(len(dataset)*0.8)

x_train = np.array([row[0]/255. for row in dataset[0:train_size]])
y_train = np.array([row[1]/255 for row in dataset[0:train_size]])

x_test = np.array([row[0]/255. for row in dataset[train_size:]])
y_test = np.array([row[1]/255 for row in dataset[train_size:]])

In [3]:
"""
np.save('x_train_seg',x_train)
np.save('y_train_seg',y_train)
np.save('x_test_seg',x_test)
np.save('y_test_seg',y_test)
"""

x_train = np.load('x_train_seg.npy')
x_test = np.load('x_test_seg.npy')
y_train = np.load('y_train_seg.npy')
y_test = np.load('y_test_seg.npy')


In [4]:
from keras.layers import *
from keras.models import Model,load_model
from keras.callbacks import ModelCheckpoint,TensorBoard
from keras.optimizers import Adam
from keras.losses import binary_crossentropy
from keras.utils import plot_model
import pydot

import tensorflow as tf
import keras.backend as K

c:\users\admin\anaconda3\envs\tensorflow-gpu\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [5]:
def dice_coef(y_true, y_pred, smooth=1):
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    
    intersection = K.sum(y_true_f * y_pred_f)
    return (2. * intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)

def dice_coef_loss(y_true, y_pred):
    return 1-dice_coef(y_true, y_pred)

def bce_dice_loss(y_true, y_pred):
    return binary_crossentropy(y_true, y_pred) + dice_coef_loss(y_true, y_pred)

In [4]:
def get_model_memory_usage(batch_size, model):
    from keras import backend as K

    shapes_mem_count = 0
    for l in model.layers:
        single_layer_mem = 1
        for s in l.output_shape:
            if s is None:
                continue
            single_layer_mem *= s
        shapes_mem_count += single_layer_mem

    trainable_count = int(np.sum([K.count_params(p) for p in set(model.trainable_weights)]))
    non_trainable_count = int(np.sum([K.count_params(p) for p in set(model.non_trainable_weights)]))

    total_memory = 4*batch_size*(shapes_mem_count + trainable_count + non_trainable_count)
    gbytes = round(total_memory / (1024 ** 3), 3)
    mbytes = round(total_memory / (1024 ** 2), 3)
    
    print('trainable_count', trainable_count, 'non_trainable_count', non_trainable_count, 'gbytes', gbytes, 'mbytes', mbytes)

def down(filters, input_):
    down_ = Conv2D(filters, (3, 3), padding='same')(input_)
    down_ = BatchNormalization(epsilon=1e-4)(down_)
    down_ = Activation('relu')(down_)
    down_ = Conv2D(filters, (3, 3), padding='same')(down_)
    down_ = BatchNormalization(epsilon=1e-4)(down_)
    down_res = Activation('relu')(down_)
    down_pool = MaxPooling2D((2, 2), strides=(2, 2))(down_)
    return down_pool, down_res

def up(filters, input_, down_):
    up_ = UpSampling2D((2, 2))(input_)
    up_ = concatenate([down_, up_], axis=3)
    up_ = Conv2D(filters, (3, 3), padding='same')(up_)
    up_ = BatchNormalization(epsilon=1e-4)(up_)
    up_ = Activation('relu')(up_)
    up_ = Conv2D(filters, (3, 3), padding='same')(up_)
    up_ = BatchNormalization(epsilon=1e-4)(up_)
    up_ = Activation('relu')(up_)
    up_ = Conv2D(filters, (3, 3), padding='same')(up_)
    up_ = BatchNormalization(epsilon=1e-4)(up_)
    up_ = Activation('relu')(up_)
    return up_

def get_unet_1024(input_shape=(None, None, 3), num_classes=1):
    inputs = Input(shape=input_shape)

    #down0b, down0b_res = down(8, inputs)
    down0a, down0a_res = down(24, inputs)
    down0, down0_res = down(64, down0a)
    down1, down1_res = down(128, down0)
    down2, down2_res = down(256, down1)
    down3, down3_res = down(512, down2)
    down4, down4_res = down(768, down3)
    
    center = Conv2D(768, (3, 3), padding='same')(down4)
    center = BatchNormalization(epsilon=1e-4)(center)
    center = Activation('relu')(center)
    center = Conv2D(768, (3, 3), padding='same')(center)
    center = BatchNormalization(epsilon=1e-4)(center)
    center = Activation('relu')(center)

    up4 = up(768, center, down4_res)
    up3 = up(512, up4, down3_res)
    up2 = up(256, up3, down2_res)
    up1 = up(128, up2, down1_res)
    up0 = up(64, up1, down0_res)
    up0a = up(24, up0, down0a_res)
    #up0b = up(8, up0a, down0b_res)

    classify = Conv2D(num_classes, (1, 1), activation='sigmoid', name='final_layer')(up0a)

    model = Model(inputs=inputs, outputs=classify)

    return model

In [5]:
model = get_unet_1024(input_shape=(None,None,3))
model.compile(loss=bce_dice_loss, optimizer=Adam(lr=1e-4), metrics=[dice_coef])
get_model_memory_usage(16, model)


trainable_count 59947081 non_trainable_count 20592 gbytes 3.577 mbytes 3662.54


In [6]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, None, None, 3 0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, None, None, 2 672         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, None, None, 2 96          conv2d_1[0][0]                   
__________________________________________________________________________________________________
activation_1 (Activation)       (None, None, None, 2 0           batch_normalization_1[0][0]      
__________________________________________________________________________________________________
conv2d_2 (

In [28]:
model_checkpoint = ModelCheckpoint('./unet1024_full_data.hdf5',save_best_only=True)
tbCallBack = TensorBoard(log_dir='./Graph/unet1024', histogram_freq=0, write_graph=True, write_images=True)


In [ ]:
history = model.fit(x_train,y_train,epochs=10,batch_size=8,validation_data = (x_test,y_test),callbacks=[model_checkpoint,tbCallBack])

Train on 1600 samples, validate on 400 samples
Epoch 1/10
 200/1600 [==>...........................] - ETA: 11:50 - loss: 1.2202 - dice_coef: 0.4026

In [31]:
out = model.predict(x_test,verbose=True)

120/120 [==============================] - 9s 78ms/step


In [6]:
model = load_model('./unet1024.hdf5',custom_objects={'bce_dice_loss':bce_dice_loss,'dice_coef':dice_coef})

In [8]:
y_pred = model.predict(x_test,verbose=1)

400/400 [==============================] - 53s 132ms/step


In [24]:
y_pred_class = (y_pred > 0.5)

In [26]:
average_dice = 0
for (im1,im2) in zip(y_pred_class,y_test):
    average_dice += dice(im1,im2)
average_dice = average_dice/y_pred_class.shape[0]

In [31]:
print ("Average dice is %f ",average_dice)

Average dice is %f  0.7947610515027125
